In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import pickle
import numpy as np

def get_text_content(a_element):
    # Extract the href attribute (URL)
    link = a_element['href']

    # Extract the text content
    address = a_element.get_text(strip=True)
    return address, f'https://www.boliga.dk/{link}'

def get_boliga(page, status=True):
    
    address_list = []
    url = f'https://www.boliga.dk/salg/resultater?searchTab=1&page={page}&sort=date-d&zipcodeFrom=1000&zipcodeTo=9990'
    try:
        response = requests.get(url, timeout=3)
        response.raise_for_status()  # Raises a HTTPError if the status is 4xx, 5xx
    except (requests.RequestException, ValueError):
        print("Network or parsing error")
        
        # Write the error pages to a text file
        with open('error_pages.txt', 'a') as txt_file:
            txt_file.write(f"{page},")
            
    else:
        soup = BeautifulSoup(response.text, 'html.parser')
        a_elements = soup.find_all('a', {'data-gtm': 'sales_address', 'class': 'text-primary fw-bolder text-start'})

        # for a_element in a_elements:
        #     pd.concat((df,get_text_content(a_element)))
        for a_element in a_elements:
            address_list.append(get_text_content(a_element))
    if page%50 == 0 and status == True or page == 1:
        print(f'Page {page} is done')
        
    # Write the error pages to a text file
    with open('pagenumber.txt', 'a') as txt_file:
        txt_file.write(f"{page},")
        
    address_df = pd.DataFrame(address_list, columns=['address', 'link'])
    return address_df


# for page in range(1, 30_000):
#     if page == 1:
#         df = get_boliga(page)
#     else:
#         df = pd.concat((df, get_boliga(page)))
#         df.to_pickle('boliga.pkl')

#     end_time = time.time()  # Stop the timer
